# 12 Epochs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/GitHub

/content/drive/MyDrive/GitHub


In [ ]:
repository = "NAS_project"

In [ ]:
%cd {repository}

/content/drive/MyDrive/GitHub/NAS_project


In [ ]:
pip install nats_bench

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install yacs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install simplejson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install xautodl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# setto una variabile d'ambiente TORCH_HOME
%env TORCH_HOME=/content/drive/MyDrive/NAS-PULITO/.torch

env: TORCH_HOME=/content/drive/MyDrive/NAS-PULITO/.torch


In [ ]:
#import delle library
import torch
import argparse
import datasets 
import nasspace
import pandas as pd
import csv
import random
import numpy as np
from timeit import default_timer as timer
import os
from scipy import stats
from nats_bench import create
import xautodl  # import this lib -- "https://github.com/D-X-Y/AutoDL-Projects", you can use pip install xautodl
from xautodl.models import get_cell_based_tiny_net

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## **Calcolo 12Hp**

In [ ]:
number_experiments = 6
results = []

arch_info = ['arch_id','time','acc']

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


GPU = '0'
seed = 1
batch_size = 128

#Per Imagenet
dataset = 'ImageNet16-120'
data_loc = "/content/drive/MyDrive/NAS/.torch/ImageNet16"

#per Cifar100
#dataset = 'cifar100'
#data_loc = "/content/drive/MyDrive/NAS-PULITO/cifar-100-python"

#per Cifar10
#dataset = 'cifar10'
#data_loc = '/content/drive/MyDrive/NAS-PULITO/cifar-10-batches-py'

save_loc = 'results'
save_string = 'naswot'
n_samples = 100
save_loc = 'results'
score_name = 'naswot'
trainval = 0
N = 10
arch_dict = []

df_config = pd.read_csv("conf.csv")
searchspace = nasspace.get_search_space(dataset, df_config.conf)
train_loader = datasets.get_data(dataset, data_loc,batch_size)




filename = f'12hpvalidation_on_{dataset}_in_{number_experiments}_experimetns_with_N={N}.csv'




for i in range(number_experiments):
  print(i)
  cumulative_time = 0
  best_accuracy = 0
  best_arch = 0
  for i in range(N):

    random_arch_id = searchspace.random_arch()
    cumulative_time += searchspace.get_training_time(random_arch_id)
    current_accuracy_12epoch = searchspace.get_12epoch_accuracy(int(random_arch_id),"x-valid",False)
    if current_accuracy_12epoch >= best_accuracy:
      best_accuracy = current_accuracy_12epoch
      best_arch = random_arch_id


  arch_dict.append({'arch_id' : best_arch,'time': cumulative_time , 'acc' : searchspace.get_final_accuracy(best_arch)})
   
  with open(filename, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = arch_info)
    writer.writeheader()
    writer.writerows(arch_dict)
  

# RESULTS

In [ ]:
validation12hp_on_cifar10 = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/12hpvalidation_on_cifar10_in_30_experimetns_with_N=10.csv")
print(validation12hp_on_cifar10.mean())
print(validation12hp_on_cifar10.std())

arch_id    7367.433333
time       1134.290214
acc          92.642889
dtype: float64
arch_id    4120.581401
time         85.741727
acc           0.945563
dtype: float64


In [ ]:
validation_on_cifar10 = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/12hpvalidation_on_cifar10_in_30_experimetns_with_N=100.csv")
print(validation_on_cifar10.mean())
print(validation_on_cifar10.std())

arch_id    11223.000000
time       11083.012635
acc           93.980000
dtype: float64
arch_id    2368.226552
time        191.042529
acc           0.257245
dtype: float64


In [ ]:
validation_on_cifar10 = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/12hpvalidation_on_cifar10_in_30_experimetns_with_N=1000.csv")
print(validation_on_cifar10.mean())
print(validation_on_cifar10.std())

arch_id      8647.700000
time       112872.632078
acc            94.096000
dtype: float64
arch_id    4056.389121
time        990.106775
acc           0.280786
dtype: float64


In [ ]:
validation12hp_on_cifar100 = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/12hpvalidation_on_cifar100_in_30_experimetns_with_N=10.csv")
print(validation12hp_on_cifar100.mean())
print(validation12hp_on_cifar100.std())

arch_id    7109.933333
time       2220.828279
acc          68.940556
dtype: float64
arch_id    4029.204131
time        227.749349
acc           1.450888
dtype: float64


In [ ]:
validation12hp_on_cifar100 = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/12hpvalidation_on_cifar100_in_30_experimetns_with_N=100.csv")
print(validation12hp_on_cifar100.mean())
print(validation12hp_on_cifar100.std())

arch_id     7417.733333
time       22124.033004
acc           71.216778
dtype: float64
arch_id    5044.048216
time        603.518087
acc           1.082671
dtype: float64


In [ ]:
validation12hp_on_cifar100 = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/12hpvalidation_on_cifar100_in_30_experimetns_with_N=1000.csv")
print(validation12hp_on_cifar100.mean())
print(validation12hp_on_cifar100.std())

arch_id      8063.200000
time       220177.507414
acc            72.115111
dtype: float64
arch_id    4400.056139
time       1779.909480
acc           0.890533
dtype: float64


In [ ]:
validation12hp_on_ImageNet = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/12hpvalidation_on_ImageNet16-120_in_30_experimetns_with_N=10.csv")
print(validation12hp_on_ImageNet.mean())
print(validation12hp_on_ImageNet.std())

arch_id    8015.833333
time       6594.244457
acc          41.453518
dtype: float64
arch_id    4454.955753
time        437.897686
acc           2.754028
dtype: float64


In [ ]:
validation12hp_on_ImageNet = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/12hpvalidation_on_ImageNet16-120_in_30_experimetns_with_N=100.csv")
print(validation12hp_on_ImageNet.mean())
print(validation12hp_on_ImageNet.std())

arch_id     7334.866667
time       66670.543294
acc           44.839630
dtype: float64
arch_id    4429.547838
time       1740.130253
acc           1.508987
dtype: float64


In [ ]:
validation12hp_on_ImageNet = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/12hpvalidation_on_ImageNet16-120_in_30_experimetns_with_N=1000.csv")
print(validation12hp_on_ImageNet.mean())
print(validation12hp_on_ImageNet.std())

arch_id      8226.933333
time       665714.631970
acc            46.210741
dtype: float64
arch_id    4210.014268
time       4956.537684
acc           0.585487
dtype: float64
